### 1. 简介

- **提出背景**：LDA在小样本问题上的表现不佳，其特征值和特征向量的估计会变得不稳定，导致在计算投影矩阵时，投影方向不准确，影响分类效果。尤其是样本量小于特征数时，会导致LDA的$S_w$不可逆。因此作者基于最大边界准则(MMC)，从几何的角度上降维后最大化类间的（平均）边缘，以及核化（非线性）的版本。

- **算法思想**：最大边缘准则的思想是通过降维操作，将样本点投影到一个低维子空间中，使得不同类别的样本点在投影后的空间中尽可能远离彼此，从而增大它们之间的间隔（边缘）。这样做的目的是为了在降维后的空间中更容易找到一个决策边界，能够有效地区分不同类别。这个目标函数是一个关于投影矩阵的函数，它考虑了类间的距离和类内的距离。具体地，我们可以定义一个评估样本点之间距离的指标，例如欧氏距离，通过调整投影矩阵，使得不同类别之间的距离最大化，同时保持同一类别内样本之间的距离尽量小。
- **算法比对**：相比于传统的PCA和LDA，其特点有：
  - 与PCA相比，MMC考虑了类别信息，因此在分类问题上的表现更好。
  - 与LDA相比，其注重提高数据的可分离性，MMC有正交化约束，不需要计算$S_w$，且有助于保持分布的形状，在小样本问题上表现得更好。


### 2. 算法介绍和流程

#### 2.1. MMC介绍
##### 2.1.1. 几何直观

在特征空间中，每个样本点表示为一个向量，不同类别的样本点被分配到不同的区域。最大边缘准则的思想是通过降维操作，将样本点投影到一个低维子空间中，使得不同类别的样本点在投影后的空间中尽可能远离彼此，从而增大它们之间的间隔（边缘）。这样做的目的是为了在降维后的空间中更容易找到一个决策边界，能够有效地区分不同类别。

##### 2.1.2. 数学表达

在MMC中，通过优化一个目标函数来实现最大化边缘。这个目标函数是一个关于投影矩阵的函数，它考虑了类间的距离和类内的距离。具体地，可以定义一个评估样本点之间距离的指标，例如欧氏距离，通过调整投影矩阵，使得不同类别之间的距离最大化，同时保持同一类别内样本之间的距离尽量小。

##### 2.1.3. 算法优势（与PCA、LDA的联系）

- 与传统的PCA和LDA相比，它更注重于提高数据的可分离性。
- 相对于LDA，MMC不会受到小样本问题的影响，因此更稳定。
- 相较于PCA，MMC更能够保留数据中的类别信息，因此在数据可分性方面效果更好。

#### 2.2. MMC流程

### 翻译
在执行降维时，我们希望找到一个从测量空间 $\mathcal{M}$ 到某个特征空间 $\mathcal{F}$ 的（线性或非线性）映射，使得转换后 $J$ 最大化。在本节中，我们讨论如何找到一个最优的线性特征提取器。在下一节中，我们将其推广到非线性情况。
考虑一个线性映射 $\mathbf{W}\in\mathcal{R}^{D\times d}$，其中 $D$ 和 $d$ 分别是转换前和转换后的数据维度。我们希望最大化
$$J(\mathbf{W})=\operatorname{tr}\left(\mathbf{S}_b^W-\mathbf{S}_w^W\right)$$
其中，$\mathbf{S}_b^W$ 和 $\mathbf{S}_w^W$ 分别是特征空间 $\mathcal{F}$ 中的类间散布矩阵和类内散布矩阵。由于 $\mathbf{W}$ 是一个线性映射，很容易得出 $\mathbf{S}_{b}^{W}=\mathbf{W}^{T}\mathbf{S}_b\mathbf{W}$ 和 $\mathbf{S}_{w}^{W}=\mathbf{W}^{T}\mathbf{S}_w\mathbf{W}$。所以我们有
$$J(\mathbf{W})=\mathrm{tr}(\mathbf{W}^T(\mathbf{S}_b-\mathbf{S}_w)\mathbf{W}).$$
(8)
在这个公式中，我们有自由度可以乘以 $\mathbf{W}$ 的某个非零常数。因此，我们额外要求 $\mathbf{W}$ 由单位向量构成，即 $\mathbf{W}=[\mathbf{w}_{1},\mathbf{w}_{2},\ldots,\mathbf{w}_{d}]$ 并且 $\mathbf{w}_k^{T}\mathbf{w}_k=1$。这意味着我们需要解决以下约束优化问题：
$$\begin{aligned}
&\max\quad\sum_{k=1}^d\mathbf{w}_k^T(\mathbf{S}_b-\mathbf{S}_w)\mathbf{w}_k\\
&\mathrm{subject~to}\quad\mathbf{w}_k^T\mathbf{w}_k-1=0\quad k=1,\ldots,d.
\end{aligned}$$
需要注意的是，我们也可以使用其他约束。例如，我们可以要求 $\mathrm{tr}(\mathbf{W}^{T}\mathbf{S}_w\mathbf{W})=1$ 并最大化 $\mathrm{tr}(\mathbf{W}^{T}\mathbf{S}_b\mathbf{W})$。很容易证明，在这种约束下最大化MMC实际上会得到LDA。唯一的区别在于它涉及到一个约束优化，而传统的LDA是一个无约束优化。使用约束 $\mathbf{W}^{T}\mathbf{W}=1$ 的动机是，它可以避免计算 $\mathbf{S}_w$ 的逆，从而避免潜在的小样本问题。
为了求解上述优化问题，我们可以引入一个拉格朗日函数
$$\mathcal{L}(\mathbf{w}_k,\lambda_k)=\sum_{k=1}^d\mathbf{w}_k^T(\mathbf{S}_b-\mathbf{S}_w)\mathbf{w}_k-\lambda_k\left(\mathbf{w}_k^T\mathbf{w}_k-1\right) \quad(9)$$
其中 $\lambda_k$ 是拉格朗日乘子。拉格朗日函数 $\mathcal{L}$ 需要对 $\lambda_k$ 和 $\mathbf{w}_k$ 进行最大化。在驻点处，$\mathcal{L}$ 对 $\mathbf{w}_k$ 的导数必须消失
$$\frac{\partial\mathcal{L}(\mathbf{w}_k,\lambda_k)}{\partial\mathbf{w}_k}=\left((\mathbf{S}_b-\mathbf{S}_w)-\lambda_k\mathbf{I}\right)\mathbf{w}_k=0\quad k=1,\ldots,d$$
这导致
(11)
$$(\mathbf{S}_b-\mathbf{S}_w)\mathbf{w}_k=\lambda_k\mathbf{w}_k\quad k=1,\ldots,d$$
这意味着 $\lambda_k$ 是 $\mathbf{S}_b-\mathbf{S}_w$ 的特征值，$\mathbf{w}_k$ 是相应的特征向量。因此
$$\begin{aligned}
J(\mathbf{W})&=\sum_{k=1}^d\mathbf{w}_k^T(\mathbf{S}_b-\mathbf{S}_w)\mathbf{w}_k\\
&=\sum_{k=1}^d\lambda_k\mathbf{w}_k^T\mathbf{w}_k=\sum_{k=1}^d\lambda_k.
\end{aligned}$$
(12)
因此，当 $\mathbf{W}$ 由 $\mathbf{S}_b-\mathbf{S}_w$ 的前 $d$ 个最大特征向量组成时，$J(\mathbf{W})$ 达到最大化。由于使用了正交约束，因此不需要计算 $\mathbf{S}_w$ 的逆，从而可以轻松避免小样本问题，并且有助于保持分布的形状。

### 4. 论文摘译

#### 4.1. 摘要

在模式识别中，特征提取技术被广泛应用于降低数据的维度并增强判别信息。主成分分析（PCA）和线性判别分析（LDA）是最流行的两种线性降维方法。然而，PCA在提取最具判别力的特征方面效果不佳，而LDA由于小样本问题而不稳定。在本文中，我们提出了一些基于最大边缘准则（MMC）的新特征提取器（包括线性和非线性）。从几何上看，基于MMC的特征提取器在降维后最大化类间的（平均）边缘。结果表明，MMC比PCA能更好地表示类的可分离性。作为与LDA的联系，我们可以通过引入一些约束条件从MMC推导出LDA。通过使用其他约束条件，我们建立了一种新的线性特征提取器，该提取器不受小样本问题的影响，这个问题已知会导致LDA的严重稳定性问题。本文还建立了这种线性特征提取器的核化（非线性）对应版本。我们的大量实验表明，这些新的特征提取器是有效的、稳定的和高效的。

#### 4.2. 最大边缘准则介绍

假设我们有以下经验数据：
$$(\mathbf{x}_1,y_1),\ldots,(\mathbf{x}_n,y_n)\in\mathcal{X}\times\{\mathcal{C}_1,\ldots,\mathcal{C}_c\}.$$
其中，域 $\mathcal{X}\in\mathcal{R}^D$ 是样本 $\mathbf{x}_i$ 所来自的非空集合。$y_i$ 被称为标签或目标。通过研究这些样本，我们希望预测某些新模式 $\mathbf{x}\in\mathcal{X}$ 的标签 $y\in\{\tilde{\mathcal{C}}_{1},\ldots,\tilde{\mathcal{C}}_{c}\}$。换句话说，我们选择 $y$ 使得 $(\mathbf{x},y)$ 在某种意义上与训练样本相似。为此，需要使用某种度量来评估相似性或不相似性。我们希望在降维后尽可能保留这种相似性/不相似性的信息，即将 $\mathbf{x}$ 从 $\mathcal{R}^D$ 转换到 $\mathcal{R}^d$，其中 $d \ll D$。
如果使用某种距离度量来测量不相似性，我们希望同类样本彼此接近，不同类样本彼此远离。因此，一个好的特征提取器应该在转换后最大化类间的距离。因此，我们可以将特征提取准则定义为：
$$ J = \frac{1}{2} \sum_{i=1}^{c} \sum_{j=1}^{c} p_i p_j d(\mathcal{C}_i, \mathcal{C}_j). $$
我们称 (2) 为最大边缘准则（MMC）。它实际上是所有类间边缘的总和。
可以使用均值向量之间的距离作为类间距离，即：
$$ d(\mathcal{C}_i, \mathcal{C}_j) = d(\mathbf{m}_i, \mathbf{m}_j), $$
其中 $\mathbf{m}_i$ 和 $\mathbf{m}_j$ 分别是类 $\mathcal{C}_i$ 和类 $\mathcal{C}_j$ 的均值向量。然而，(3) 并不适用，因为它忽略了类的散布。即使均值向量之间的距离很大，如果两类的分布很大并且彼此重叠，也不容易分开。考虑到类的散布，我们将类间距离（或边缘）定义为：
$$ d(\mathcal{C}_i, \mathcal{C}_j) = d(\mathbf{m}_i, \mathbf{m}_j) - s(\mathcal{C}_i) - s(\mathcal{C}_j), $$
其中 $s(\mathcal{C}_i)$ 是类 $\mathcal{C}_i$ 的散布的一种度量。在统计学中，我们通常使用广义方差 $|\mathbf{S}_i|$ 或总方差 $\mathrm{tr}(\mathbf{S}_i)$ 来测量数据的散布，其中 $\mathbf{S}_i$ 是类 $\mathcal{C}_i$ 的协方差矩阵。在本文中，我们使用总方差 $\mathrm{tr}(\mathbf{S}_i)$ 因为它易于分析。总方差的弱点在于它完全忽略了协方差结构。注意，通过使用总方差/广义方差，表达式 (4) 测量的是两个类之间的“平均边缘”，而支持向量机（SVMs）使用最小边缘。
使用 (4) 和 $s(\mathcal{C}_i) = \mathrm{tr}(\mathbf{S}_i)$，我们可以将 (2) 分解为两部分：
$$ \begin{aligned}
J &= \frac{1}{2} \sum_{i=1}^{c} \sum_{j=1}^{c} p_i p_j \left( d(\mathbf{m}_i, \mathbf{m}_j) - \mathrm{tr}(\mathbf{S}_i) - \mathrm{tr}(\mathbf{S}_j) \right) \\
&= \frac{1}{2} \sum_{i=1}^{c} \sum_{j=1}^{c} p_i p_j d(\mathbf{m}_i, \mathbf{m}_j) - \frac{1}{2} \sum_{i=1}^{c} \sum_{j=1}^{c} p_i p_j \left( \mathrm{tr}(\mathbf{S}_i) + \mathrm{tr}(\mathbf{S}_j) \right).
\end{aligned} $$
第二部分可以简化为 $\mathrm{tr}(\mathbf{S}_w)$：
$$ \begin{aligned}
&\frac{1}{2} \sum_{i=1}^{c} \sum_{j=1}^{c} p_i p_j \left( \mathrm{tr}(\mathbf{S}_i) + \mathrm{tr}(\mathbf{S}_j) \right) \\
&= \sum_{i=1}^{c} p_i \mathrm{tr}(\mathbf{S}_i) = \mathrm{tr}\left( \sum_{i=1}^{c} p_i \mathbf{S}_i \right) = \mathrm{tr}(\mathbf{S}_w).
\end{aligned} $$
通过使用欧几里得距离，我们还可以将第一部分简化为 $\mathrm{tr}(\mathbf{S}_b)$：
$$ \begin{aligned}
&\frac{1}{2} \sum_{i=1}^{c} \sum_{j=1}^{c} p_i p_j d(\mathbf{m}_i, \mathbf{m}_j) \\
&= \frac{1}{2} \sum_{i=1}^{c} \sum_{j=1}^{c} p_i p_j (\mathbf{m}_i - \mathbf{m}_j)^T (\mathbf{m}_i - \mathbf{m}_j) \\
&= \frac{1}{2} \sum_{i=1}^{c} \sum_{j=1}^{c} p_i p_j (\mathbf{m}_i - \mathbf{m} + \mathbf{m} - \mathbf{m}_j)^T (\mathbf{m}_i - \mathbf{m} + \mathbf{m} - \mathbf{m}_j).
\end{aligned} $$
通过展开，可以使用以下事实简化上述等式：
$$ \sum_{i=1}^{c} p_j (\mathbf{m}_{j} - \mathbf{m})^T (\mathbf{m}_{j} - \mathbf{m}). $$
最终得到：
$$ J = \mathrm{tr}(\mathbf{S}_b - \mathbf{S}_w). $$
因为 $\mathrm{tr}(\mathbf{S}_b)$ 测量的是类均值向量的总方差，较大的 $\mathrm{tr}(\mathbf{S}_b)$ 表明类均值向量在一个大空间中分散。另一方面，较小的 $\mathrm{tr}(\mathbf{S}_w)$ 表明每个类的分布较小。因此，较大的 $J$ 表示来自同一类的样本彼此接近，但来自不同类的样本彼此远离。因此，这一准则可能比 PCA 更好地表示类的可分离性。回想一下，PCA 试图在线性变换后最大化总方差 $\mathrm{tr}{(\mathbf{S}_t)}$，但具有大类内散布的数据集即使具有小的类间散布，也可能具有大的总散布，因为 $\mathbf{S}_t = \mathbf{S}_b + \mathbf{S}_w$。显然，这种数据不容易分类。
